In [1]:
from pyspark.sql import SparkSession
from delta import *

builder = (
    SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/nil/anaconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nil/.ivy2/cache
The jars for the packages stored in: /home/nil/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e014a90c-fa6b-4312-af38-332e901ed429;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 249ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

In [7]:
spark.sql("show tables").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [8]:
DeltaTable.createIfNotExists(spark).tableName("employee_table").addColumn(
    "emp_id", "INT"
).addColumn("emp_name", "STRING").addColumn("gender", "STRING").addColumn(
    "salary", "INT"
).addColumn(
    "Dept", "STRING"
).property(
    "description", "table created for demo purpose"
).location(
    "/mnt/sda3/iNeuron/Data Engineering/pySpark/Delta_Lake/delta_lake_02"
).execute()

In [10]:
spark.sql(
    "insert into employee_table values(100,'Stephen','M',2000,'IT'),(200,'Philipp','M',8000,'HR'),(300,'Laura','F',6000,'SALES')"
)

DataFrame[]

In [11]:
spark.sql("SELECT * FROM employee_table").show(truncate=False)

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary|Dept |
+------+--------+------+------+-----+
|300   |Laura   |F     |6000  |SALES|
|100   |Stephen |M     |2000  |IT   |
|200   |Philipp |M     |8000  |HR   |
+------+--------+------+------+-----+



### forPath

In [12]:
deltaInstance1 = DeltaTable.forPath(
    sparkSession=spark, path="./Delta_Lake/delta_lake_02"
)

In [13]:
deltaInstance1.toDF().show(truncate=False)

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary|Dept |
+------+--------+------+------+-----+
|300   |Laura   |F     |6000  |SALES|
|100   |Stephen |M     |2000  |IT   |
|200   |Philipp |M     |8000  |HR   |
+------+--------+------+------+-----+



In [14]:
type(deltaInstance1.toDF())

pyspark.sql.dataframe.DataFrame

#### DML Operation

- delete operation

In [15]:
# 1st way
spark.sql("delete from employee_table where emp_id=100")

DataFrame[num_affected_rows: bigint]

In [16]:
spark.sql("select * from employee_table").show()

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary| Dept|
+------+--------+------+------+-----+
|   300|   Laura|     F|  6000|SALES|
|   200| Philipp|     M|  8000|   HR|
+------+--------+------+------+-----+



In [17]:
# 2nd way (using instance)
deltaInstance1.delete("emp_id=200")

In [18]:
deltaInstance1.toDF().show(truncate=False)

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary|Dept |
+------+--------+------+------+-----+
|300   |Laura   |F     |6000  |SALES|
+------+--------+------+------+-----+



### forName

In [19]:
deltaInstance2 = DeltaTable.forName(
    sparkSession=spark, tableOrViewName="employee_table"
)

In [20]:
deltaInstance2.toDF().show()

+------+--------+------+------+-----+
|emp_id|emp_name|gender|salary| Dept|
+------+--------+------+------+-----+
|   300|   Laura|     F|  6000|SALES|
+------+--------+------+------+-----+



In [22]:
spark.sql("DESCRIBE HISTORY employee_table").show(truncate=False)

+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                         |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                  

In [23]:
deltaInstance2.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                         |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                  